In [3]:
#! /bin/python3

In [ ]:
# import packages
!pip install faker
!pip install names
import json
import pandas
import ipywidgets as widgets
import faker
from IPython.display import display

In [2]:
# import os_core
from os_core.users import users
from os_core.specimen import specimen
from os_core.collection_protocoll import collection_protocol
from os_core.collecttion_protocol_registration import collection_protocol_registration
from os_core.csv_bulk import csv_bulk
from os_core.csv_export import CSV_exporter
from os_core.jsons import Json_factory


In [3]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    import os
    from IPython.core.display import display, HTML
    link = "http://localhost:8888/edit/work/" + csv_file_name
    df.to_csv(csv_file_name, index=False)
    display(HTML("<a href={} target=_blank>Download/Edit</a>".format(link)))
   
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [4]:
# import os_util
from os_util.bulk_operations import bulk_operations
from os_util.export_operations import Export_OP
from os_util.query_util import query_util

In [5]:
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
#base_url='http://localhost:9000' # input()
base_url='http://biobank.silicolab.bibbox.org'
base_url2= 'http://biobank-7-2.silicolab.bibbox.org' 
base_url+='/openspecimen/rest/ng'
base_url2+='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter
The whole URL for API Calls is: http://biobank.silicolab.bibbox.org/openspecimen/rest/ng


In [6]:
# Loginname and Pasword of the API-User, recommended is an extra API User
print('Loginname of the user:')
loginname='admin'#input()
print('Password of the user:')
password='Login@123'#input()

Loginname of the user:
Password of the user:


The next step is to choose the entity which will be created or updated. The following schemas are in Openspecimen available: (Name, openspecimen entity). One have to write the openspecimen entity.
('Collection Protocol', 'cp')
('Collection Protocol Registration', 'cpr')
('User', 'user')
('User Roles', 'userRoles')
('Site', 'site')
('Shipments', 'shipment')
('Institute', 'institute')
('Distribution Protocol Requirement', 'dpRequirement')
('Distribution Protocol', 'distributionProtocol')
('Distribution Order', 'distributionOrder')
('Containers', 'storageContainer')
('Container Type', 'storagecontainerType')
('Container Shipment', 'containerShipment')
('Collection Protocol Event', 'cpe')
('Master Specimen', 'masterSpecimen')
('Participant', 'participant')
('Specimen Requirement', 'sr')
('Visit', 'visit')
('Specimen Aliquot', 'specimenAliquot')
('Specimen Derivate', 'specimenDerivative')
('Specimen Disposal', 'specimenDisposal')
('Consent', 'consent')

In [7]:
# Working schemes so far 
schema=widgets.Dropdown(
    options= [(), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collection Protocol Registration', 'cpr'),
              ('User', 'user'), ('User Roles', 'userRoles'), ('Site', 'site'), ('Shipments', 'shipment'), 
              ('Institute', 'institute'), ('Distribution Protocol Requirement', 'dpRequirement'), 
              ('Distribution Protocol', 'distributionProtocol'), ('Distribution Order', 'distributionOrder'), 
              ('Containers', 'storageContainer'), ('Container Type', 'storagecontainerType'),
              ('Container Shipment', 'containerShipment'), ('Collection Protocol Event', 'cpe'),
              ('Master Specimen', 'masterSpecimen'), ('Participant', 'participant'), ('Specimen Requirement', 'sr'),
              ('Visit', 'visit'), ('Specimen Aliquot', 'specimenAliquot'), 
              ('Specimen Derivate', 'specimenDerivative'),('Specimen Disposal', 'specimenDisposal'),
              ('Consent', 'consent')],
    value=('Specimen', 'specimen'),
    description='Entity:')

operation = widgets.Dropdown(
    options = ["CREATE",  'UPDATE'],
              value="CREATE",
              description='Operation:')

print('Choose your Entity:')
display(schema)
print('Choose Operation:')
display(operation)

Choose your Entity:


Dropdown(description='Entity:', index=2, options=((), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'),…

Choose Operation:


Dropdown(description='Operation:', options=('CREATE', 'UPDATE'), value='CREATE')

In [8]:
print("Selected Schema:")
print(schema.value)
sel_schema = schema.value[1]
print(sel_schema)
print("Selected Operation:")
sel_op = operation.value
print(sel_op)

Selected Schema:
('Collection Protocol Registration', 'cpr')
cpr
Selected Operation:
CREATE


In [9]:
#initialize csv_bulk, bulk_operations and CSV_exporter

csv_files=csv_bulk(base_url2,(loginname, password))
bulk_op=bulk_operations(base_url2, (loginname, password))
# csv_export = Export_OP(base_url, (loginname, password))
csv_export_7_2 = Export_OP(base_url2, (loginname, password))

In [19]:
# get necessary cp information since we fetch data from a different Openspecimen Client and aggregate Fields on
# another one

cp_tools_source = collection_protocol(base_url, (loginname, password))
cp_tools_target = collection_protocol(base_url2, (loginname, password))

# get source cp info
cps_source = cp_tools_source.get_all_collection_protocols()
cps_dict_source = {}
for i, item in enumerate(cps_source):
    
    print("Cp Title:")
    print(item["title"])
    print("Cp ID:", item["id"], "\n")
    
    cps_dict_source[i] = [item["title"], item["id"]]

# get target cp info
cps_target = cp_tools_target.get_all_collection_protocols()
cps_dict_target = {}
for i, item in enumerate(cps_target):
    
    print("Cp Title:")
    print(item["title"])
    print("Cp ID: ",item["id"], "\n")
    
    cps_dict_target[i] = [item["title"], item["id"]]

# choose source CP/ Check if matching is correct
# print("Source CP: ", cps_dict_source[1])
# cp_id_source = cps_dict_source[1][1]
# cp_source = cp_tools_source.get_cp_def(cp_id_source)
# cp_def_target1 = cp_tools_source.get_collection_protocol(cp_id_source)

mult_cp = []

# choose target CP / Check if matching is correct
print("Target CP: ", cps_dict_target[1])
cp_id_target = cps_dict_target[1][1]
cp_target = cp_tools_target.get_collection_protocol(cp_id_target)
cp_def_target = cp_tools_target.get_cp_def(cp_id_target)
mult_cp.append([cp_id_target, cp_def_target])

# choose another target CP / Check if matching is correct
print("Target CP 2: ", cps_dict_target[2])
cp_id_target_2 = cps_dict_target[2][1]
cp_target_2 = cp_tools_target.get_collection_protocol(cp_id_target)
cp_def_target_2 = cp_tools_target.get_cp_def(cp_id_target_2)
mult_cp.append([cp_id_target_2, cp_def_target_2])



Cp Title:
Main collection of Clinical-biological Biobank at INAB, comprising of all the samples.
Cp ID: 9 

Cp Title:
Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence
Cp ID: 10 

Cp Title:
Hellenic Biobank for Parkinson's Disease
Cp ID: 7 

Cp Title:
Main collection of Clinical-biological Biobank at INAB, comprising of all the samples.
Cp ID:  16 

Cp Title:
Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence
Cp ID:  17 

Cp Title:
Hellenic Biobank for Parkinson's Disease
Cp ID:  18 

Target CP:  ['Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence', 17]
Target CP 2:  ["Hellenic Biobank for Parkinson's Disease", 18]


In [21]:
# Download Template file, Export File for data aggregation
# Use the template Downloads to check which data needs to present 

template = csv_files.get_template(schemaname=sel_schema)
filename = sel_schema +'_template.csv'
csv_download_link(template, filename)
# download cp CSV for aggregating fields:
export_cps = []
if len(mult_cp) != 1:
    for cp_data in mult_cp:
        cp_id = cp_data[0]
        export_cps.append(csv_export_7_2.export_file(sel_schema, cp_id))
else:
    export = csv_export_7_2.export_file(sel_schema, cp_id_target)

# print("First CP:")
# print(export_cps[0])
# print()
# print("2nd CP:")
# print(export_cps[1])
# print(len(export_cps))
    

In [28]:
import numpy as np

# in case of multiple targets; just an example
if export_cps:
    for h, cp in enumerate(export_cps):
        age_array = []
        # print(cp)
        # print(cp["Date Of Birth"])
        for date in cp["Date Of Birth"]:
            age_array.append(2020 - float(date.split("-")[-1]))
        age_array = np.array(age_array)
        print("Age high: ",np.max(age_array))
        print("Age low: ", np.min(age_array))
        print("Age mean: ", age_array.mean())
        cp_data = mult_cp[h]
        for i, item in enumerate(cp_data[1]["extensionDetail"]["attrs"]):
            if cp_data[1]["extensionDetail"]["attrs"][i]["name"] == "AgeLow":
                cp_data[1]["extensionDetail"]["attrs"][i]["value"] = str(int(np.min(age_array)))
                cp_data[1]["extensionDetail"]["attrs"][i]["displayValue"] = str(np.min(age_array))
            elif cp_data[1]["extensionDetail"]["attrs"][i]["name"] == "AgeHigh":
                cp_data[1]["extensionDetail"]["attrs"][i]["value"] = str(int(np.max(age_array)))
                cp_data[1]["extensionDetail"]["attrs"][i]["displayValue"] = str(np.max(age_array))
        # print("Current cp update:") 
        # print(cp["extensionDetail"]["attrs"])
        # input()
        
# in case of only one target
else: 
    age_array = []
    # crude example of age calculation
    for item in export["Date Of Birth"]:
        age_array.append(2020 - float(item.split("-")[-1]))

    age_array = np.array(age_array)
    print("Age high: ",np.max(age_array))
    print("Age low: ", np.min(age_array))
    print("Age mean: ", age_array.mean())

    #rewrite our custom Aggregtion fields (there is a better way to do this)
    for i, item in enumerate(cp_target["extensionDetail"]["attrs"]):

        if cp_target["extensionDetail"]["attrs"][i]["name"] == "AgeLow":
            cp_target["extensionDetail"]["attrs"][i]["value"] = str(int(np.min(age_array)))
            cp_target["extensionDetail"]["attrs"][i]["displayValue"] = str(np.min(age_array))
        elif cp_target["extensionDetail"]["attrs"][i]["name"] == "AgeHigh":
            cp_target["extensionDetail"]["attrs"][i]["value"] = str(int(np.max(age_array)))
            cp_target["extensionDetail"]["attrs"][i]["displayValue"] = str(np.max(age_array))

    # left out for brevity
    # print(cp_target["extensionDetail"]["attrs"])


Age high:  89.0
Age low:  4.0
Age mean:  46.812613156306575
Age high:  98.0
Age low:  0.0
Age mean:  49.096676737160124


In [29]:
#update CP at Target with fields set; returns error in case of

if export_cps:
    for cp_data in mult_cp:
        cp_id = cp_data[0]
        cp_def = cp_data[1]
        updated_cp_json = cp_tools_target.update_collection_protocol(cp_id, json.dumps(cp_def))
        # print(updated_cp_json)
        # input()
else:
    updated_cp_json = cp_tools_target.update_collection_protocol(cp_id_target, json.dumps(cp_target))
    # print(updated_cp_json)

In [23]:
#Open file from working dir and run upload

### old miabis version says Study Name ---> ignore begin
export = export.rename(columns={'Study Short Title': 'CP Short Title'})
print(export.columns)
### ignore end
transfer_data =  export.to_csv(index=False)

uplo=bulk_op.bulk_import(file=transfer_data,
                         filename='upload_cpr.csv', 
                         schemaname=sel_schema, 
                         operation=sel_op, dateformat="MM-dd-yyyy"
                        )
print(uplo)

Index(['Identifier', 'CP Short Title', 'PPID', 'Registration Date',
       'Registration Site', 'External Subject ID', 'Activity Status', 'Reason',
       'First Name', 'Last Name', 'Middle Name', 'Birth Date', 'Death Date',
       'Sex', 'Race#1', 'Vital Status', 'Ethnic Group#1',
       'Personal Identification Number', 'eMPI', 'PMI#1#Site Name',
       'PMI#1#MRN', 'MIABIS participant#Proband',
       'MIABIS participant#Relation with Proband',
       'MIABIS participant#Categories of data collected#1',
       'MIABIS participant#Death Cause', 'MIABIS participant#Survey Data'],
      dtype='object')
"Identifier","CP Short Title","PPID","Registration Date","Registration Site","External Subject ID","Activity Status","Reason","First Name","Last Name","Middle Name","Birth Date","Death Date","Sex","Race#1","Vital Status","Ethnic Group#1","Personal Identification Number","eMPI","PMI#1#Site Name","PMI#1#MRN","MIABIS participant#Proband","MIABIS participant#Relation with Proband","MIABIS pa

ParserError: Error tokenizing data. C error: EOF inside string starting at row 394

In [32]:
# Example update participant data using query API 

if export_cps:
    for cp_data in mult_cp:
        cp_id = cp_data[0]
        qry = query_util(base_url=base_url2, auth=(loginname, password))
        qry_str = 'select Participant.id where CollectionProtocol.id = {}'.format(cp_id)
        exqry = qry.create_aql(cpid=cp_id, aql= qry_str)
        cprID = exqry['rows']
        part = collection_protocol_registration(base_url=base_url2, auth=(loginname, password))
        fake = faker.Faker()
        start_date = datetime.date(year=2010, month=1, day=1)
        for ID in cprID:
            params = {"participant":{"birthDate": str(fake.date_between(start_date='-90y', end_date='-5y'))},
                "registrationDate": str(fake.date_between(start_date=start_date, end_date='today'))}
            params = json.dumps(params)
            partici = part.update_participant(ID[0], params)
        print(partici)

NameError: name 'faker' is not defined

In [13]:
# Update some records (operation='UPDATE')
upload_=pandas.read_csv('update_specimen.csv',index_col=False)
uplo=bulk_op.bulk_import(file=upload_.to_csv(index=False),filename='upload_specimen.csv', schemaname=schema,operation='UPDATE')
print(uplo)

  Specimen Label OS_IMPORT_STATUS  OS_ERROR_MESSAGE
0       Jupyter1          SUCCESS               NaN
1       Jupyter3          SUCCESS               NaN
2       Jupyter4          SUCCESS               NaN
